In [ ]:
%pip install transformers datasets evaluate tensorflow ipywidgets scikit-learn numpy pandas


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

In [81]:
df = pd.read_json('https://raw.githubusercontent.com/Wikidepia/SQuAD-id/refs/heads/master/data/train-SQuAD-id.json')

In [82]:
squad_data = []

for index, row in df.iterrows():
    entry = row['data']
    title = entry['title']
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            id_ = qa['id']
            if qa['is_impossible']:
                answers = {'answer_start': [], 'text': []}
            else:
                answers = {
                    'answer_start': [answer['answer_start'] for answer in qa['answers']],
                    'text': [answer['text'] for answer in qa['answers']]
                }
            squad_data.append({
                'answers': answers,
                'context': context,
                'id': id_,
                'question': question,
                'title': title
            })

squad_df = pd.DataFrame(squad_data)
squad_df.head(5)

,answers,context,id,question,title
0,"{'answer_start': [299], 'text': ['pada akhir 1...",Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ /...,56be85543aeaaa14008c9063,Kapan Beyonce mulai populer?,Beyoncé
1,"{'answer_start': [238], 'text': ['menyanyi dan...",Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ /...,56be85543aeaaa14008c9065,Di bidang apa Beyonce bersaing ketika dia tumb...,Beyoncé
2,"{'answer_start': [564], 'text': ['2003']}",Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ /...,56be85543aeaaa14008c9066,Kapan Beyonce meninggalkan Destiny's Child dan...,Beyoncé
3,"{'answer_start': [190], 'text': ['Houston, Tex...",Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ /...,56bf6b0f3aeaaa14008c9601,Di kota dan negara bagian manakah Beyonce tumbuh?,Beyoncé
4,"{'answer_start': [304], 'text': ['akhir 1990-a...",Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ /...,56bf6b0f3aeaaa14008c9602,Pada dekade berapa Beyonce menjadi terkenal?,Beyoncé


In [83]:
squad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114825 entries, 0 to 114824
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   answers   114825 non-null  object
 1   context   114825 non-null  object
 2   id        114825 non-null  object
 3   question  114825 non-null  object
 4   title     114825 non-null  object
dtypes: object(5)
memory usage: 4.4+ MB


In [88]:
# Split the processed dataset into train, test, and validation sets
train_size = 0.7  # 70% for training
test_size = 0.2   # 20% for testing
val_size = 0.1    # 10% for validation

# Split the dataset
train_indices, test_val_indices = train_test_split(range(len(processed_dataset)), train_size=train_size, random_state=42)
test_indices, val_indices = train_test_split(test_val_indices, test_size=val_size/(test_size + val_size), random_state=42)

# Create the train, test, and validation datasets
train_dataset = processed_dataset.select(train_indices)
test_dataset = processed_dataset.select(test_indices)
val_dataset = processed_dataset.select(val_indices)

# Verify the split
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 80377
Test dataset size: 22965
Validation dataset size: 11483
